In [15]:
import numpy as np
from PyLTSpice import SimCommander, RawRead

high_side = True
low_side = False

def frequencytoltpulse(frequency, side: bool):
    deadband = 8  # ns
    period = 1 / (frequency / 1E9)  # Convert to Gigahertz to ensure the period will be in nanoseconds
    if side == low_side:
        return f"PULSE(3.3 0 0n 1n 1n {period + (deadband * 2)}n {period * 2}n)"
    else:
        return f"PULSE(0 3.3 {deadband}n 1n 1n {period}n {period * 2}n)"

fsw = 100E3
L = 10E-6
C = 47E-6

# select spice model
LTC = SimCommander("LTFiles/EPC23102_Mine.asc")
# set default arguments
LTC.set_component_value('VHSin', frequencytoltpulse(fsw, high_side))
LTC.set_component_value('VLSin', frequencytoltpulse(fsw, low_side))

Using LTspice installed in : 'C:\Users\treys\AppData\Local\Programs\ADI\LTspice\LTspice.exe' 
Creating Netlist
The Netlist was successfully created


In [20]:
run_netlist_file = f'LTFiles/F{fsw}_L{L}_C{C}'
LTC.run(run_filename=f'{run_netlist_file}.net')

Wed Apr 26 10:49:13 2023 : Starting simulation 2


<RunTask(sim2, started 28792)>

In [33]:
LTR = RawRead(f'{run_netlist_file}.raw')
print(LTR.get_trace_names())
vin = np.array(LTR.get_trace('V(vin)'))
vout = np.array(LTR.get_trace('V(vout)'))
iin = np.array(LTR.get_trace('I(Vvin)'))
pin = vin * iin
print(np.average(pin))

v_ripple = np.ptp(vout)
print(v_ripple)

Reading file with encoding  utf_16_le
File contains 31 traces, reading 31
Binary RAW file with Normal access
['time', 'V(hsin)', 'V(lsin)', 'V(vout)', 'V(sw)', 'V(vboot)', 'V(vdd5)', 'V(vin)', 'V(vdd_int)', 'I(C1)', 'I(C2)', 'I(C3)', 'I(C4)', 'I(L1)', 'I(I1)', 'I(Vhsin)', 'I(Vlsin)', 'I(Vvdd5)', 'I(Vvin)', 'Ix(u1:ENB)', 'Ix(u1:HSIN)', 'Ix(u1:LSIN)', 'Ix(u1:VDRV)', 'Ix(u1:AGND)', 'Ix(u1:PGND)', 'Ix(u1:RBOOT)', 'Ix(u1:RDRV)', 'Ix(u1:SW)', 'Ix(u1:VBOOT)', 'Ix(u1:VDD)', 'Ix(u1:VIN)']
-5.832082
0.9329524
